In [8]:
import os

In [2]:
# + SubVert

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
# config = tf.ConfigProto()  
# config.gpu_options.allow_growth=True  
# session = tf.Session(config=config)
 
# KTF.set_session(session)

Using TensorFlow backend.


In [10]:
import csv
import datetime
import numpy as np
import time
import json
import itertools
import random
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [11]:
import nltk 
from nltk.tokenize import word_tokenize
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_fscore_support

In [46]:
import keras
from keras.utils.np_utils import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.layers import Layer, InputSpec
from keras import initializers #keras2
from keras.utils.vis_utils import plot_model
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *

from tensorflow.keras.utils import Sequence


In [12]:
from datetime import datetime
import time
def trans2tsp(timestr):
    return int(time.mktime(datetime.strptime(timestr, '%m/%d/%Y %I:%M:%S %p').timetuple()))

In [13]:
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [14]:
def shuffle(pn,labeler,pos):
    index=np.arange(pn.shape[0])
    pn=pn[index]
    labeler=labeler[index]
    pos=pos[index]
    
    for i in range(pn.shape[0]):
        index=np.arange(npratio+1)
        pn[i,:]=pn[i,index]
        labeler[i,:]=labeler[i,index]
    return pn,labeler,pos

In [15]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [7]:
news_set = set()
news = []
with open('/data/Recommend/MIND/MINDsmall_train/news.tsv', 'r') as f:
    for line in f:
        data = line.split('\t')
        news_id = data[0]
        if news_id not in news_set:
            news.append(line)
            news_set.add(news_id)
with open('/data/Recommend/MIND/MINDsmall_dev/news.tsv') as f:
    for line in f:
        data = line.split('\t')
        news_id = data[0]
        if news_id not in news_set:
            news.append(line)
            news_set.add(news_id)
# with open('/data/Recommend/MIND/MINDlarge_test/news.tsv') as f:
#     for line in f:
#         data = line.split('\t')
#         news_id = data[0]
#         if news_id not in news_set:
#             news.append(line)
#             news_set.add(news_id)

# with open('/data/Recommend/MIND/small_news.tsv', 'w') as f:
#     f.writelines(news)

print(len(news))

65238


In [13]:
#REad News

In [16]:
def read_news(path,filenames):
    news={}
    category=[]
    subcategory=[]
    news_index={}
    index=1
    word_dict={}
    word_index=1
    body_word_dict = {}
    with open(os.path.join(path,filenames)) as f:
        lines=f.readlines()
    for line in lines:
        splited = line.strip('\n').split('\t')
        doc_id,vert,subvert,title= splited[0:4]
        news_index[doc_id]=index
        index+=1
        category.append(vert)
        subcategory.append(subvert)
        title = title.lower()
        title=word_tokenize(title)
        #body = release2private[doc_id]
        #body = body.split()[:200]
        body = None
        news[doc_id]=[vert,subvert,title,body]
        for word in title:
            word = word.lower()
            if not(word in word_dict):
                word_dict[word]=word_index
                word_index+=1
        
    category=list(set(category))
    subcategory=list(set(subcategory))
    category_dict={}
    index=1
    for c in category:
        category_dict[c]=index
        index+=1
    subcategory_dict={}
    index=1
    for c in subcategory:
        subcategory_dict[c]=index
        index+=1
        
    for word in body_word_dict:
        if word in word_dict:
            continue
        if body_word_dict[word]<3:
            continue
        word_dict[word] = word_index
        word_index += 1
        
    return news,news_index,category_dict,subcategory_dict,word_dict

In [17]:
# data_root_path = '/data1/qitao/MIND-Full-Release/'
# embedding_path = '/data/data/qit/'
data_root_path = '/data/Recommend/MIND/'

In [20]:
# %time news,news_index,category_dict,subcategory_dict,word_dict = read_news(data_root_path,'docs.tsv')
%time news,news_index,category_dict,subcategory_dict,word_dict = read_news(data_root_path,'small_news.tsv')

CPU times: user 7.89 s, sys: 143 ms, total: 8.03 s
Wall time: 8.03 s


In [21]:
MAX_SENTENCE = 30

In [22]:
def get_doc_input(news,news_index,category,subcategory,word_dict):
    news_num=len(news)+1
    news_title=np.zeros((news_num,MAX_SENTENCE),dtype='int32')
    news_body=None
    news_vert=np.zeros((news_num,),dtype='int32')
    news_subvert=np.zeros((news_num,),dtype='int32')
    for key in news:    
        vert,subvert,title,body=news[key]
        doc_index=news_index[key]
        news_vert[doc_index]=category[vert]
        news_subvert[doc_index]=subcategory[subvert]
        for word_id in range(min(MAX_SENTENCE,len(title))):
            news_title[doc_index,word_id]=word_dict[title[word_id].lower()]
        
    return news_title,news_vert,news_subvert

In [24]:
news_title,news_vert,news_subvert=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict)

In [25]:
news_info = news_title

In [26]:
embedding_path = '/data/pretrained/Glove/'
def load_matrix(embedding_path,word_dict):
    embedding_matrix = np.zeros((len(word_dict)+1,300))
    have_word=[]
    with open(os.path.join(embedding_path,'glove.840B.300d.txt'),'rb') as f:
        while True:
            l=f.readline()
            if len(l)==0:
                break
            l=l.split()
            word = l[0].decode()
            if word in word_dict:
                index = word_dict[word]
                tp = [float(x) for x in l[1:]]
                embedding_matrix[index]=np.array(tp)
                have_word.append(word)
    return embedding_matrix,have_word

In [27]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)

In [23]:
#End Read News

In [24]:
# Read Session

In [29]:
def read_clickhistory(path,filename):
    
    lines = []
    userids = []
    with open(os.path.join(data_root_path,filename)) as f:
        lines = f.readlines()
        
    sessions = []
    for i in range(len(lines)):
        _,uid,eventime, click, imps = lines[i].strip().split('\t')
        if click == '':
            clicks = []
        else:
            clikcs = click.split()
        true_click = []
        for click in clikcs:
            if not click in news_index:
                continue
            true_click.append(click)
        pos = []
        neg = []
        for imp in imps.split():
            docid, label = imp.split('-')
            if label == '1':
                pos.append(docid)
            else:
                neg.append(docid)
        sessions.append([true_click,pos,neg])
    return sessions

In [30]:
# train_session = read_clickhistory(data_root_path,'train_sam2.tsv')
train_session = read_clickhistory(data_root_path,'MINDsmall_train/behaviors.tsv')

In [31]:
def read_part_clickhistory(path,filename):
    
    lines = []
    userids = []
    with open(os.path.join(data_root_path,filename)) as f:
        lines = f.readlines()
    lines = lines[:100000]
        
    sessions = []
    for i in range(len(lines)):
        _,uid,eventime, click, imps = lines[i].strip().split('\t')
        if click == '':
            clicks = []
        else:
            clikcs = click.split()
        true_click = []
        for click in clikcs:
            if not click in news_index:
                continue
            true_click.append(click)
        pos = []
        neg = []
        for imp in imps.split():
            docid, label = imp.split('-')
            if label == '1':
                pos.append(docid)
            else:
                neg.append(docid)
        sessions.append([true_click,pos,neg])
    return sessions

In [32]:
# test_session = read_part_clickhistory(data_root_path,'val_sam2.tsv')
test_session = read_part_clickhistory(data_root_path,'MINDsmall_train/behaviors.tsv')

In [29]:
ls /data/qit16/MIND-Large-Release/

ls: 无法访问/data/qit16/MIND-Large-Release/: 没有那个文件或目录


In [30]:
#Parser User

In [33]:
MAX_ALL = 50

In [34]:
def parse_user(session):
    user_num = len(session)
    user={'click': np.zeros((user_num,MAX_ALL),dtype='int32'),}
    for user_id in range(len(session)):
        tclick = []
        click, pos, neg =session[user_id]
        for i in range(len(click)):
            tclick.append(news_index[click[i]])
        click = tclick

        if len(click) >MAX_ALL:
            click = click[-MAX_ALL:]
        else:
            click=[0]*(MAX_ALL-len(click)) + click
            
        user['click'][user_id] = np.array(click)
    return user

In [35]:
train_user = parse_user(train_session)

In [36]:
test_user = parse_user(test_session)

In [35]:
#Get Training Data

In [37]:
npratio=4

In [38]:
def get_train_input(session):
    sess_pos = []
    sess_neg = []
    user_id = []
    for sess_id in range(len(session)):
        sess = session[sess_id]
        _, poss, negs=sess
        for i in range(len(poss)):
            pos = poss[i]
            neg=newsample(negs,npratio)
            sess_pos.append(pos)
            sess_neg.append(neg)
            user_id.append(sess_id)
    print(len(user_id))
    sess_all = np.zeros((len(sess_pos),1+npratio),dtype='int32')
    label = np.zeros((len(sess_pos),1+npratio))
    for sess_id in range(sess_all.shape[0]):
        pos = sess_pos[sess_id]
        negs = sess_neg[sess_id]
        sess_all[sess_id,0] = news_index[pos]
        index = 1
        for neg in negs:
            sess_all[sess_id,index] = news_index[neg]
            index+=1
        #index = np.random.randint(1+npratio)
        label[sess_id,0]=1
    user_id = np.array(user_id, dtype='int32')
    
    return sess_all, user_id, label

In [39]:
train_sess, train_user_id, train_label = get_train_input(train_session)

236344


In [40]:
def get_test_input(session):
    
    Impressions = []
    userid = []
    for sess_id in range(len(session)):
        _, poss, negs = session[sess_id]
        imp = {'labels':[],
                'docs':[]}
        userid.append(sess_id)
        for i in range(len(poss)):
            docid = news_index[poss[i]]
            imp['docs'].append(docid)
            imp['labels'].append(1)
        for i in range(len(negs)):
            docid = news_index[negs[i]]
            imp['docs'].append(docid)
            imp['labels'].append(0)
        Impressions.append(imp)
        
    userid = np.array(userid,dtype='int32')
    
    return Impressions, userid,

In [41]:
test_impressions, test_userids = get_test_input(test_session)

In [47]:
class Attention(Layer):
 
    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ',
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK',
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
 
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
 
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [48]:
MAX_SENT_LENGTH=30
MAX_SENTS=50

In [49]:
def AttentivePooling(dim1,dim2):
    vecs_input = Input(shape=(dim1,dim2),dtype='float32')
    user_vecs =Dropout(0.2)(vecs_input)
    user_att = Dense(200,activation='tanh')(user_vecs)
    user_att = keras.layers.Flatten()(Dense(1)(user_att))
    user_att = Activation('softmax')(user_att)
    user_vec = keras.layers.Dot((1,1))([user_vecs,user_att])
    model = Model(vecs_input,user_vec)
    return model

In [50]:
def get_doc_encoder(length,word_dict,title_word_embedding_matrix):

    sentence_input = Input(shape=(length,), dtype='int32')
    title_word_embedding_layer = Embedding(len(word_dict)+1, 300, weights=[title_word_embedding_matrix],trainable=True)
    word_vecs = title_word_embedding_layer(sentence_input)
    droped_vecs = Dropout(0.2)(word_vecs)
    #word_rep = Conv1D(400,kernel_size=3,activation='relu')(droped_vecs)
    word_rep = Attention(20,20)([droped_vecs]*3)
    droped_rep = Dropout(0.2)(word_rep)
    title_vec = AttentivePooling(length,400)(droped_rep)
    sentEncodert = Model(sentence_input, title_vec)
    return sentEncodert

In [52]:

class get_hir_train_generator(Sequence):
    def __init__(self,news_info, clicked_news,user_id, news_id, label, batch_size):
        self.news_info = news_info
        
        self.clicked_news = clicked_news

        self.user_id = user_id
        self.doc_id = news_id
        self.label = label
        
        
        self.batch_size = batch_size
        self.ImpNum = self.user_id.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))
    
    def __get_news(self,docids):
        news_info = self.news_info[docids]
        
        return news_info
        

    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
            
        label = self.label[start:ed]

        doc_ids = self.doc_id[start:ed]
        info = self.__get_news(doc_ids)
        
        user_ids = self.user_id[start:ed]
        clicked_ids = self.clicked_news[user_ids]
        user_info = self.__get_news(clicked_ids)


        return ([info,user_info],[label])
    
    
class get_hir_user_generator(Sequence):
    def __init__(self,news_info, clicked_news,batch_size):
        self.news_info = news_info
        
        self.clicked_news = clicked_news

        self.batch_size = batch_size
        self.ImpNum = self.clicked_news.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))

    
    def __get_news(self,docids):
        news_info = self.news_info[docids]

        return news_info
    
    
    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
            
        clicked_ids = self.clicked_news[start:ed]
        user_info = self.__get_news(clicked_ids)

        return user_info

In [53]:
def evaluate(user_scorings,news_scorings,Impressions):
    AUC = []
    MRR = []
    nDCG5 = []
    nDCG10 =[]
    for i in range(len(Impressions)):
        docids = Impressions[i]['docs']
        labels = Impressions[i]['labels']
        mean = np.mean(labels)

        uv = user_scorings[i]
        
        docids = np.array(docids,dtype='int32')
        nv = news_scorings[docids]
        score = np.dot(nv,uv)
        auc = roc_auc_score(labels,score)
        mrr = mrr_score(labels,score)
        ndcg5 = ndcg_score(labels,score,k=5)
        ndcg10 = ndcg_score(labels,score,k=10)
    
        AUC.append(auc)
        MRR.append(mrr)
        nDCG5.append(ndcg5)
        nDCG10.append(ndcg10)

    AUC = np.array(AUC)
    MRR = np.array(MRR)
    nDCG5 = np.array(nDCG5)
    nDCG10 = np.array(nDCG10)
    
    AUC = AUC.mean()
    MRR = MRR.mean()
    nDCG5 = nDCG5.mean()
    nDCG10 = nDCG10.mean()
    
    return AUC, MRR, nDCG5, nDCG10

In [54]:
def get_doc_encoder(mode):

    length = 30
    word_embedding_matrix = title_word_embedding_matrix
    sentence_input = Input(shape=(length,),dtype='int32')
    
    word_embedding_layer = Embedding(word_embedding_matrix.shape[0], 300, weights=[word_embedding_matrix],trainable=True)
    word_vecs = word_embedding_layer(sentence_input)
    droped_vecs = Dropout(0.2)(word_vecs)
    
    if mode in ['NRMS','NAML']:
        word_rep = Attention(20,20)([droped_vecs]*3)
        droped_rep = Dropout(0.2)(word_rep)
    elif mode == 'NRMS-ReLU':
        word_rep = Attention(20,20)([droped_vecs]*3)
        droped_rep = Dropout(0.2)(word_rep)
    elif mode == 'FastFormer':
        word_rep = Fastformer(20,20)([droped_vecs]*3)
        droped_rep = Dropout(0.2)(word_rep)
    elif mode in ['NAML-woRelu']:
        word_rep = Conv1D(400,3,padding='same')(droped_vecs)
        droped_rep = Dropout(0.2)(word_rep)
    elif mode in ['NAML','LSTUR']:
        word_rep = Conv1D(400,3,padding='same',activation='relu')(droped_vecs)
        droped_rep = Dropout(0.2)(word_rep)
    elif mode in ['NAML-Dense']:
        word_rep = Conv1D(400,3,padding='same',activation='relu')(droped_vecs)
        word_rep = Dense(400)(word_rep)
        droped_rep = Dropout(0.2)(word_rep)      
        
    title_vec = AttentivePooling(length,400)(droped_rep)

    sentEncodert = Model(sentence_input, title_vec)
    return sentEncodert

In [55]:
def get_user_encoder(mode):
    user_vecs_input = Input(shape=(50,400))    
    click_mask_input = Input(shape=(50,))
    
    user_vecs = Dropout(0.2)(user_vecs_input)

    if mode=='NRMS':
        user_vecs = Attention(20,20)([user_vecs]*3)
        user_vec = AttentivePooling(50,400)(user_vecs)
    elif mode in ['LSTUR','GRU']:
        user_vec = GRU(400)(user_vecs)
    elif mode in ['NAML']:
        #user_vecs = Dense(400)(user_vecs)
        user_vec = AttentivePooling(50,400)(user_vecs)
    elif mode in ['NAML-Dense']:
        user_vec = AttentivePooling(50,400)(user_vecs)
        user_vec = Dense(400)(user_vec)
        
    return Model(user_vecs_input,user_vec)

In [56]:
def create_model(mode):
    
    MAX_LENGTH = 30  
    news_encoder = get_doc_encoder(mode,)

    user_encoder = get_user_encoder(mode)
    
    clicked_title_input = Input(shape=(50,MAX_LENGTH,), dtype='int32')

    title_inputs = Input(shape=(1+npratio,MAX_LENGTH,),dtype='int32') 

    clicked_title_vecs = TimeDistributed(news_encoder)(clicked_title_input)
    news_vecs = TimeDistributed(news_encoder)(title_inputs)
    
    news_vecs = Dropout(0.2)(news_vecs)
    clicked_title_vecs = Dropout(0.2)(clicked_title_vecs)

    user_vec = user_encoder(clicked_title_vecs)
    

    scores = keras.layers.Dot(axes=-1)([news_vecs,user_vec])

    
    
    logits = keras.layers.Activation(keras.activations.softmax,name = 'recommend')(scores)     

    model = Model([title_inputs,clicked_title_input,],logits) # max prob_click_positive
    model.compile(loss=['categorical_crossentropy'],
                  optimizer=Adam(lr=0.0001,),
                  metrics=['acc'])

    
    return model,news_encoder,user_encoder

In [57]:
train_generator = get_hir_train_generator(news_title,train_user['click'],train_user_id,train_sess,train_label,32)

In [65]:
mode = 'NAML'
model,news_encoder,user_encoder = create_model(mode)
Res = []
model.fit_generator(train_generator,epochs=1,verbose=1)
for i in range(3):
    model.fit_generator(train_generator,epochs=1,verbose=1)
    news_scoring = news_encoder.predict(news_title,verbose=True)
    test_user_generator = get_hir_user_generator(news_scoring,test_user['click'],32)
    user_scoring = user_encoder.predict_generator(test_user_generator,verbose=True)
    g = evaluate(user_scoring,news_scoring,test_impressions)
    print(g)

ValueError: Exception encountered when calling layer "attention_2" (type Attention).

in user code:

    File "/tmp/ipykernel_1048174/2751447696.py", line 57, in call  *
        A = K.permute_dimensions(A, (0,3,2,1))
    File "/home/gene/anaconda3/envs/torch/lib/python3.8/site-packages/keras/backend.py", line 3288, in permute_dimensions
        return tf.compat.v1.transpose(x, perm=pattern)

    ValueError: Dimension must be 5 but is 4 for '{{node attention_2/transpose_7}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32](attention_2/truediv, attention_2/transpose_7/perm)' with input shapes: [?,20,30,20,30], [4].


Call arguments received:
  • x=['tf.Tensor(shape=(None, 30, 300), dtype=float32)', 'tf.Tensor(shape=(None, 30, 300), dtype=float32)', 'tf.Tensor(shape=(None, 30, 300), dtype=float32)']

In [52]:
g

(0.6776793480631789,
 0.3298717043234166,
 0.3649363442750286,
 0.4291449716730456)

In [64]:
import tensorflow as tf
tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

2022-04-21 09:47:52.618079: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


False